In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import spacy
from collections import Counter
import kagglehub
from spacy.matcher import PhraseMatcher
import pandas as pd

nltk.download('stopwords')
nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

IT_SKILLS = [
    'python', 'django', 'flask', 'sql', 'nosql', 'mongodb', 'postgresql',
    'java', 'javascript', 'react', 'angular', 'vue', 'node.js', 'typescript',
    'flutter', 'dart', 'swift', 'kotlin', 'ios', 'android', 'aws', 'azure',
    'gcp', 'docker', 'kubernetes', 'machine learning', 'deep learning',
    'tensorflow', 'pytorch', 'pandas', 'numpy', 'scikit-learn', 'git',
    'html', 'css', 'rest api', 'graphql', 'agile', 'scrum', 'ci/cd', 'linux'
]
path = kagglehub.dataset_download("kshitizregmi/jobs-and-job-description")
df_postings = pd.read_csv(f"{path}/job_title_des.csv")
df_postings.head()
print(df_postings.isnull().mean() * 100)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Using Colab cache for faster access to the 'jobs-and-job-description' dataset.
Unnamed: 0         0.0
Job Title          0.0
Job Description    0.0
dtype: float64


In [2]:
df_postings.head()

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [3]:
def extract_skills_spacy(text, nlp, matcher):
    """Clean text and extract skills using spaCy PhraseMatcher"""
    if pd.isnull(text) or not isinstance(text, str):
        return []
    doc = nlp(text.lower())
    matches = matcher(doc)
    skills = [doc[start:end].text for _, start, end in matches]
    skills = list(set(nlp(skill)[0].lemma_ for skill in skills if skill))
    if not skills and isinstance(text, str):
        title = text.lower()
        if 'ios' in title:
            return ['swift', 'ios']
        elif 'django' in title:
            return ['django', 'python']
        elif 'flutter' in title:
            return ['flutter', 'dart']
        elif 'machine learning' in title:
            return ['machine learning', 'python']
    return skills

In [4]:
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp.make_doc(skill) for skill in IT_SKILLS]
matcher.add("SKILLS", patterns)

In [5]:
print("Extracting skills...")
df_postings['skills_list'] = df_postings['Job Description'].apply(lambda x: extract_skills_spacy(x, nlp, matcher))

print("Sample skills_list:")
print(df_postings[['Job Title', 'skills_list']].head())

if 'job_skills' in df_postings.columns:
    print("Combining with job_skills...")
    df_postings['skills_list'] = df_postings.apply(
        lambda row: list(set(row['skills_list'] + (
            [skill.strip() for skill in row['job_skills'].split(',')]
            if isinstance(row['job_skills'], str) else []
        ))),
        axis=1
    )

Extracting skills...
Sample skills_list:
              Job Title                                        skills_list
0     Flutter Developer                                          [flutter]
1      Django Developer                [flask, python, sql, linux, django]
2      Machine Learning  [tensorflow, python, java, deep, machine, pyto...
3         iOS Developer                                               [io]
4  Full Stack Developer     [react, vue, angular, html, javascript, agile]


In [8]:
def get_common_skills(group):
    try:
        all_skills = []
        for skills in group:
            if isinstance(skills, list):
                all_skills.extend(skills)
        if not all_skills:
            return []
        skill_counts = Counter(all_skills).most_common(5)
        return [skill for skill, count in skill_counts]
    except Exception as e:
        print(f"Error in get_common_skills: {e}")
        return []

print("Aggregating skills by job title...")
try:
    skills_by_title = df_postings.groupby('Job Title')['skills_list'].apply(get_common_skills).reset_index()
    skills_by_title.columns = ['job_title', 'common_skills']
except Exception as e:
    print(f"Error during groupby: {e}")
    print("Check 'skills_list' content:")
    print(df_postings['skills_list'].apply(lambda x: type(x)).value_counts())
    exit()
try:
    sample_counts = df_postings['Job Title'].value_counts().reset_index()
    sample_counts.columns = ['job_title', 'sample_count']
    skills_by_title = pd.merge(skills_by_title, sample_counts, on='job_title')
except Exception as e:
    print(f"Error during merge: {e}")
    exit()

Aggregating skills by job title...


In [10]:
print(f"Saved common skills for {len(skills_by_title)} job titles to 'it_jobs_common_skills.csv'")
print("\nSample output:")
print(skills_by_title.head())

Saved common skills for 15 job titles to 'it_jobs_common_skills.csv'

Sample output:
                job_title                          common_skills  sample_count
0       Backend Developer     [javascript, css, html, java, git]           147
1  Database Administrator      [sql, linux, aw, python, mongodb]           139
2         DevOps Engineer        [aw, linux, python, ci, docker]           155
3        Django Developer  [django, python, javascript, git, aw]           152
4       Flutter Developer     [flutter, android, io, dart, java]           155
